# Document the Data Sources
* [Coingecko python code](https://github.com/man-c/pycoingecko)

In [8]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize classes

In [9]:
import os
import sys
import configparser
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.assets.etl import cryptoMCExtractor as crypto
# from utils.modules.etl.load import noSQLwls as nosql
from utils.modules.etl.load import sparkNoSQLwls as nosql

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    crypto = importlib.reload(crypto)
    nosql = importlib.reload(nosql)

clsNoSQL = nosql.NoSQLWorkLoads(desc="get crypto macket capitalization data")
''' optional - if not specified class will use the default values '''
prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
              }
clsMC = crypto.CryptoMarkets(
    desc='get crypto macket capitalization data', **prop_kwargs)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All assets-module etl-packages in function-CryptoMarket imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
All functional FILEWORKLOADS-libraries in LOAD-package of ETL-module imported successfully!
FileWorkLoads Class initialization complete
[Error]CryptoMarket function <__init__> name 'config' is not defined

Class initialization and load complete!


## Read catalog data
* Distinct API categories: "asset.metadata", "historic.prices", "latest.prices"

In [28]:
from datetime import datetime
from bson.json_util import dumps

__now_dt__ = str(datetime.utcnow().isoformat())
__as_type__ = 'dict'
__database__ = "tip-asset-metadata"
__collection__ = "coingecko.asset.list"
__find__ = {"source":
            {"$regex":'coin'},
#             "symbol":{"$in":['mee']},
            'asset.isactive':{"$lte":0.0},
#             "asset.isactive":{"$gt" : 0}
           }

clsNoSQL.connect={'DBAUTHSOURCE':__database__}
_data = clsNoSQL.read_documents(
    as_type=__as_type__,
    db_name = __database__,
    db_coll=__collection__,
    doc_find=__find__
)
if len(_data) < 5:
    print(dumps(_data, indent=2))
else:
    print("Retrieved %d documents" %len(_data))

Retrieved 3262 documents


## Update the Crypto Metadata List

In [3]:
_data_owner='coingecko'

_results, _collection = clsMC.update_asset_metadata(data_owner=_data_owner)

print(_collection)
print(len(_results))

Total 10848 documents, successful insert count = 1585 & modify count = 9263
coingecko.asset.list
10848


## Insert/Update new datasource

In [47]:
__data__ = [
## CMC
#     {
#         "owner":'coinmarketcap',
#         "category":'historic.prices',
#         "resource.info":'https://coinmarketcap.com/api/documentation/v1/#tag/cryptocurrency',
#         "api.url":'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/historical',
#         "api.endpoint":'/cryptocurrency/quotes/historical',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.headers.X-CMC_PRO_API_KEY":'2f2845f0-581b-46f7-9a27-51d507826675',
#         "api.parameters.localization" :'false',
#         "api.parameters.date" :'{date}',
#         "api.parameters.cryptocurrency_type" :'all',
#         "api.parameters.start" :'1',
#         "api.parameters.limit":'5000',
#         "api.parameters.convert":'USD',
#         "api.parameters.sort":'symbol',
#     },
#     {
#         "owner":'coinmarketcap',
#         "category":'latest.prices',
#         "resource.info":'https://coinmarketcap.com/api/documentation/v1/#tag/cryptocurrency',
#         "api.url":'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest',
#         "api.endpoint":'/cryptocurrency/*',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.headers.X-CMC_PRO_API_KEY":'2f2845f0-581b-46f7-9a27-51d507826675',
#         "api.parameters.localization" :'false',
#         "api.parameters.start" :'1',
#         "api.parameters.limit":'5000',
#         "api.parameters.convert":'USD',
#         "api.parameters.sort":'symbol',
#     },
#     {
#         "owner":'coinmarketcap',
#         "category":'asset.metadata',
#         "resource.info":'https://coinmarketcap.com/api/documentation/v1/#tag/cryptocurrency',
#         "api.url":'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map',
#         "api.endpoint":'/api/v3/coins/list',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.X-CMC_PRO_API_KEY":'2f2845f0-581b-46f7-9a27-51d507826675',
#         "api.parameters.localization" :'',
#         "api.parameters.listing_status" :'active',
#         "api.parameters.start" :'1',
#         "api.parameters.limit":'5000',
#         "api.parameters.sort":'cmc_rank',
#     },
## COINGEKO
#     {
#         "owner":'coingecko',
#         "category":'historic.prices',
#         "resource.info":'https://www.coingecko.com/en/api/documentation',
#         "api.url":'https://api.coingecko.com/api/v3/coins/{id}/market_chart/range',
#         "api.endpoint":'/coins/{id}/history',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.parameters.localization" :'false',
#         "api.parameters.id" :'',
#         "api.parameters.days":'',
#         "api.parameters.vs_currency":'usd',
#         "api.parameters.interval" :'',
#     },
#     {
#         "owner":'coingecko',
#         "category":'latest.prices',
#         "resource.info":'https://www.coingecko.com/en/api/documentation',
#         "api.url":'https://api.coingecko.com/api/v3/coins/markets',
#         "api.endpoint":'/coins/markets',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.parameters.localization" :'false',
#         "api.parameters.per_page" :'250',
#         "api.parameters.page" :'1',
#         "api.parameters.price_change_percentage" :'1h,24h,7d,14d,30d',
#         "api.parameters.order":'market_cap_desc',
#         "api.parameters.vs_currency":'USD',
#         "api.parameters.sparkline" :'false',
#         "api.parameters.sort":'symbol',
#     },
#     {
#         "owner":'coingecko',
#         "category":'asset.metadata',
#         "resource.info":'https://www.coingecko.com/en/api/documentation',
#         "api.url":'https://api.coingecko.com/api/v3/coins/list',
#         "api.endpoint":'/coins/list',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.parameters.include_platform" :'true',
#     },
]
_uids = ['owner','category']
_database = "tip-data-sources"
_collection = "marketcap.api"

# __keys__ = []
clsNoSQL.connect={'DBAUTHSOURCE':_database}
_collection = clsNoSQL.write_documents(
    db_name=_database,
    db_coll=_collection,
    data=__data__,
    uuid_list=_uids
)
# print([x for x in _collection])

Total 1 documents, duccessful insert count = 0 & modify count = 1


## Scripts to update with mongos

In [ ]:
import pandas as pd

_fname = os.path.join('/home/nuwan/workspace/rezaware/',
                      'wrangler/data/assets/etl/tmp/',
                      'assets_mcap_gt_100mil.csv')
_asset_list = pd.read_csv(_fname)['asset_symbol'].tolist()

In [15]:
db.coingecko.asset.list.updateMany(
   {
       "source" :  {"$eq":'coingecko'},
       "name":{"$in":[]},
   },
   [
     { "$set": 
      { "asset.isactive": -1.0 },
     }
   ],
    {
        "upsert":true
    }
)

NameError: name 'db' is not defined

In [ ]:
db.coingecko.asset.list.updateMany(
   {
       "source" :  {"$eq":'coingecko'},
       "asset.isactive": {"$eq":0.0},
   },
   [
     { "$set": 
      { "asset.isactive": 1.0 },
     }
   ],
    {
        "upsert":true
    }
)

In [ ]:
db.marketcap.api.update(
   { "category" :  'crypto.metadata'},
   [
     { "$set": 
      { "api.parameters.localization": "" },
     }
   ],
    {
        "upsert":true
    }
)